<a href="https://colab.research.google.com/github/gkmravisekarms/Cloud-Computing-GCP-/blob/main/Collaborative-Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check Java 

In [ ]:
! java -version

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


Install PySpark

In [2]:
# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=c3fbf65813a8420dc32dd334242b541120ccde337aa9d9d0affd625eb520db9f
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


After installation, we can create a Spark session and check its information.

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession

In [4]:
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
# Check Spark Session Information
spark

We can also test the installation by importing a Spark library.

In [6]:
# Import a Spark function from library
from pyspark.sql.functions import col

Import libraries

In [7]:
#https://grouplens.org/datasets/movielens/

In [8]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

Initiate spark session

In [9]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

1. Load data

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
!ls -l /content/drive/MyDrive/

total 3035
drwx------ 2 root root    4096 Nov 16 15:31 'Colab Notebooks'
-rw------- 1 root root     143 Mar 23  2022 'Copy of CS540 - Project Group List.gsheet'
-rw------- 1 root root     143 Nov 17 04:08  Gafterdisconnection.gslides
drwx------ 2 root root    4096 Nov 17 00:16  GCollaborative-filtering
-rw------- 1 root root  494429 Nov 16 02:42  gmovies.csv
-rw------- 1 root root 2483721 Nov 16 02:45  gratings.csv
-rw------- 1 root root  118658 Nov 16 02:48  gtags.csv
-rw------- 1 root root     143 Nov 17 03:07  GWeek9-HW.gslides


In [12]:
movies = spark.read.csv("/content/drive/MyDrive/gmovies.csv",header=True)

In [13]:
ratings = spark.read.csv("/content/drive/MyDrive/gratings.csv",header=True)

In [14]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [15]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [16]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



Calculate sparsity

In [17]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

In [18]:
# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

In [19]:
# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

In [20]:
# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


Interpret ratings

In [21]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [22]:
# Group data by userId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



Build Out An ALS Model

In [23]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [24]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

In [25]:
# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

In [26]:
# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

Tell Spark how to tune your ALS model

In [27]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [28]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

In [29]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


Build your cross validation pipeline

In [30]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [31]:
# Confirm cv was built
print(cv)

CrossValidator_f90cad514f07


Best Model and Best Model Parameters

In [32]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

In [33]:
#Extract best model from the cv model above
best_model = model.bestModel

In [34]:
# Print best_model
print(type(best_model))

<class 'pyspark.ml.recommendation.ALSModel'>


In [35]:
# Complete the code below to extract the ALS model parameters
print("**Best Model**")

**Best Model**


In [36]:
# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

  Rank: 50


In [37]:
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())


  MaxIter: 10


In [38]:
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

  RegParam: 0.15


In [39]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8678589181737388


In [41]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0|  3.464672|
|   148|   4896|   4.0| 3.4670408|
|   148|   4993|   3.0|   3.52035|
|   148|   7153|   3.0| 3.4120665|
|   148|   8368|   4.0| 3.5930173|
|   148|  40629|   5.0| 3.2363656|
|   148|  50872|   3.0| 3.7235408|
|   148|  60069|   4.5| 3.6385822|
|   148|  69757|   3.5|  3.519815|
|   148|  72998|   4.0|  3.232824|
|   148|  81847|   4.5| 3.5208135|
|   148|  98491|   5.0| 3.7861571|
|   148| 115617|   3.5| 3.4901662|
|   148| 122886|   3.5| 3.4427116|
|   463|    296|   4.0| 4.1831555|
|   463|    527|   4.0|  3.736365|
|   463|   2019|   4.0| 3.9875493|
|   471|    527|   4.5| 3.8419898|
|   471|   6016|   4.0| 3.9658365|
|   471|   6333|   2.5|  3.255078|
+------+-------+------+----------+
only showing top 20 rows



Make Recommendations

In [42]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3379, 5.7355146...|
|     3|[{70946, 4.866154...|
|     5|[{5490, 4.5791554...|
|     6|[{3925, 4.8140335...|
|     9|[{3379, 4.935473}...|
|    12|[{42730, 5.655360...|
|    13|[{33649, 4.978508...|
|    15|[{3379, 4.527062}...|
|    16|[{3379, 4.614239}...|
|    17|[{3379, 5.1721935...|
+------+--------------------+



In [43]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   3379|5.7355146|
|     1|  33649|5.5564117|
|     1|   5490|  5.47559|
|     1| 171495|5.4036317|
|     1|   5328| 5.361952|
|     1|   5416| 5.361952|
|     1|   3951| 5.361952|
|     1|  78836|5.3408904|
|     1|   5915| 5.309985|
|     1|   8477|5.3042073|
+------+-------+---------+



Do the recommendations make sense?

Lets merge movie name and genres to th recommendation matrix for interpretability.

In [44]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  67618|   100|  5.15273|Strictly Sexual (...|Comedy|Drama|Romance|
|  33649|   100| 5.095666|  Saving Face (2004)|Comedy|Drama|Romance|
|   3379|   100|5.0508056| On the Beach (1959)|               Drama|
|  45503|   100|4.9322023|Peaceful Warrior ...|               Drama|
|  42730|   100| 4.919008|   Glory Road (2006)|               Drama|
|  74282|   100| 4.916703|Anne of Green Gab...|Children|Drama|Ro...|
| 117531|   100|4.9119444|    Watermark (2014)|         Documentary|
| 179135|   100|4.9119444|Blue Planet II (2...|         Documentary|
|   7071|   100|4.9119444|Woman Under the I...|               Drama|
|  84273|   100|4.9119444|Zeitgeist: Moving...|         Documentary|
+-------+------+---------+--------------------+--------------------+



In [45]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

